# Push/Pull to AWS
* Training DF files
* then push other data

In [1]:
import pandas as pd
import boto3
import numpy as np
import os
from os import listdir
from os.path import isfile, join
from tqdm import tqdm_notebook
import warnings
from pathlib import Path

warnings.filterwarnings("ignore")
#set path directory

#load access key
HOME = os.path.expanduser('~')
KEYPATH = "SWEMLv2.0/AWSaccessKeys.csv"
ACCESS = pd.read_csv(f"{HOME}/{KEYPATH}")

#start session
SESSION = boto3.Session(
    aws_access_key_id=ACCESS['Access key ID'][0],
    aws_secret_access_key=ACCESS['Secret access key'][0],
)
S3 = SESSION.resource('s3')
#AWS BUCKET information
BUCKET_NAME = 'national-snow-model'
BUCKET = S3.Bucket(BUCKET_NAME)


## Training DF regional files

In [4]:
#Upload nearest-snotel dictionaries
regionlist = ['SouthernRockies', 'Southwest', 'Northwest']
output_res = 300

for region in regionlist:
    print(region)
    head_folder = f"SWEMLv2.0/data/ASO/{region}/"
    head_folder_dir = f"{HOME}/{head_folder}"

    #Get files only
    files = [f for f in listdir(head_folder_dir) if isfile(join(head_folder_dir, f))]
    for file in tqdm_notebook(files):
        S3.meta.client.upload_file(Filename= f"{head_folder_dir}{file}", Bucket=BUCKET_NAME, Key=f"{head_folder}{file}")

    #get list of directories to download
    dirs = [ f.path for f in os.scandir(head_folder_dir) if f.is_dir() ]
    for dir in dirs:
        print(f"Folder name: {dir.split('300M_Resolution/')[-1]}")
        awsfolderpath = f"SWEMLv2.0{dir.split('SWEMLv2.0')[-1]}"
        headfolderpath = f"{HOME}/{awsfolderpath}"
        files = [f for f in listdir(headfolderpath) if isfile(join(headfolderpath, f))]
        for file in tqdm_notebook(files):
             S3.meta.client.upload_file(Filename= f"{headfolderpath}/{file}", Bucket=BUCKET_NAME, Key=f"{awsfolderpath}/{file}")
            # print(f"{headfolderpath}/{file}")

        # #Get subfolders
        # subdirs = [ f.path for f in os.scandir(headfolderpath) if f.is_dir() ]
        # for dir in subdirs:
        #     print(f"Folder name: {dir.split('300M_Resolution/')[-1]}")
        #     awsfolderpath2 = f"SWEMLv2.0{dir.split('SWEMLv2.0')[-1]}"
        #     headfolderpath2 = f"{HOME}/{awsfolderpath2}"
        #     files = [f for f in listdir(headfolderpath2) if isfile(join(headfolderpath2, f))]
        #     for file in tqdm_notebook(files):
        #         S3.meta.client.upload_file(Filename= f"{headfolderpath2}/{file}", Bucket=BUCKET_NAME, Key=f"{awsfolderpath2}/{file}")



        




SouthernRockies


0it [00:00, ?it/s]

Folder name: /home/rjohnson18/SWEMLv2.0/data/ASO/SouthernRockies/Raw_ASO_Data


  0%|          | 0/19 [00:00<?, ?it/s]

Folder name: /home/rjohnson18/SWEMLv2.0/data/ASO/SouthernRockies/ASO_xml


...

  0%|          | 0/2 [00:00<?, ?it/s]

Folder name: /home/rjohnson18/SWEMLv2.0/data/ASO/Northwest/Processed_300M_SWE


  0%|          | 0/2 [00:00<?, ?it/s]

Folder name: /home/rjohnson18/SWEMLv2.0/data/ASO/Northwest/300M_SWE_parquet


  0%|          | 0/2 [00:00<?, ?it/s]

## Get files

In [5]:
#Upload nearest-snotel dictionaries
regionlist = ['SouthernRockies', 'Southwest', 'Northwest']
output_res = 300

#Get data
for region in regionlist:
    path = f"SWEMLv2.0/data/ASO/{region}/"

    #Make directory if it does not exist
    if not os.path.exists(f"{HOME}/Test/{path}"):
        print("Path not present, making")
        os.makedirs(f"{HOME}/{path}", exist_ok=True)
    
    #Identify the potential files to download
    files = [objects.key for objects in BUCKET.objects.filter(Prefix=path)]
    
    for file in tqdm_notebook(files):
        #Make directory if it does not exist
        if not os.path.exists(f"{HOME}/{path}"):
            print("Path not present, making")
        os.makedirs( os.path.dirname(f"{HOME}/{file}"), exist_ok=True)
       
        #check to see if the file is there
        if os.path.exists(f"{HOME}/{file}") == False:
            print('Downloading file')
            S3.Bucket(BUCKET_NAME).download_file(file, f"{HOME}/{file}")

        else:
            print('File already present...')


Path not present, making


  0%|          | 0/66 [00:00<?, ?it/s]

File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already pre

  0%|          | 0/459 [00:00<?, ?it/s]

File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already pre

  0%|          | 0/8 [00:00<?, ?it/s]

File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...
File already present...


In [ ]:
os.path.dirname(f"{HOME}/Test/{file}")